This notebook is for [Kaggle Russian Normalization challenge](https://www.kaggle.com/competitions/text-normalization-challenge-russian-language).

In order to reproduce the results one is necessary to download `ru_train.csv` file trom the challenge website and put it alongside the notebook.

In [1]:
PATHES = {
    "load": "ru_train.csv",
    "save": "/home/jovyan/data/kaggle.jsonl"
}

I use `Replaces` class as list of changes have been made upon the text.

In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd())))

In [3]:
from replaces import Replace, Replaces


Replaces.from_sequences(
    "мама мыла раму с мылом".split(),
    "мама раму уронила".split(),
    False
)

E|мама
R|мыла => 
E|раму
R|смылом => уронила

One is possible to construct Replaces object out from list of dicts so some kind of serialization could be done easily here

In [4]:
Replaces([{"text_from": "a", "text_to": "a"}, {"text_from": "b", "text_to": "c"}])

E|a
R|b => c

Parse kaggle train file

In [5]:
from collections import defaultdict
from tqdm import tqdm
import csv


data = defaultdict(dict)
with open(PATHES["load"]) as f:
    reader = csv.reader(f)
    next(reader, None)  # ['sentence_id', 'token_id', 'class', 'before', 'after']
    for row in tqdm(reader):
        data[int(row[0])][int(row[1])] = {
            "class": row[2],
            "before": row[3],
            "after": row[4],
        }
len(data), data[0]

10574516it [00:08, 1181950.33it/s]


(761436,
 {0: {'class': 'PLAIN', 'before': 'По', 'after': 'По'},
  1: {'class': 'PLAIN', 'before': 'состоянию', 'after': 'состоянию'},
  2: {'class': 'PLAIN', 'before': 'на', 'after': 'на'},
  3: {'class': 'DATE',
   'before': '1862 год',
   'after': 'тысяча восемьсот шестьдесят второй год'},
  4: {'class': 'PUNCT', 'before': '.', 'after': '.'}})

Quick check on are tokens indices ok

In [6]:
for sent in data.values():
    if len(sent) == len(set(sent)) == max(sent) + 1:
        continue
    print(sent)
    break

Reformat it

In [7]:
data = [{
    "sentence_id": sent_id,
    "tokens": [token for i_token, token in sorted(sent.items(), key=lambda x: x[0])]
} for sent_id, sent in tqdm(data.items())]
data[0]

100%|██████████| 761436/761436 [00:03<00:00, 219834.88it/s]


{'sentence_id': 0,
 'tokens': [{'class': 'PLAIN', 'before': 'По', 'after': 'По'},
  {'class': 'PLAIN', 'before': 'состоянию', 'after': 'состоянию'},
  {'class': 'PLAIN', 'before': 'на', 'after': 'на'},
  {'class': 'DATE',
   'before': '1862 год',
   'after': 'тысяча восемьсот шестьдесят второй год'},
  {'class': 'PUNCT', 'before': '.', 'after': '.'}]}

Check on what classes are there

In [8]:
from itertools import chain
set(chain(*[[_["class"] for _ in sent["tokens"]] for sent in data]))

{'CARDINAL',
 'DATE',
 'DECIMAL',
 'DIGIT',
 'ELECTRONIC',
 'FRACTION',
 'LETTERS',
 'MEASURE',
 'MONEY',
 'ORDINAL',
 'PLAIN',
 'PUNCT',
 'TELEPHONE',
 'TIME',
 'VERBATIM'}

One is necessary to polish spaces of tokens

In [9]:
import re


re_trans = re.compile(r"_(trans|latin) *")
examples = []
for elem in tqdm(data):
    for key in ("before", "after"):
        for token in elem["tokens"]:
            if "_" in token[key]:  # quicker check to speedup
                token[key] = re.sub(re_trans, "", str(token[key]))
        for i1, (t1, t2) in enumerate(zip(elem["tokens"], elem["tokens"][1:])):
            if t1[key] in ("(", "«"):
                t1[key] = " " + t1[key]
                t2[key] = t2[key].strip()
            elif t2["class"] == "PUNCT":
                if t2[key] == "—":
                    t1[key] += " "
                else:
                    pass
            elif t1["class"] == t2["class"] == "ORDINAL" and t2[key].startswith("—"):
                pass
            elif t1["class"] == "VERBATIM" or t2["class"] == "VERBATIM" and t1["class"] != "PUNCT":
                pass
            else:
                t1[key] += " "


100%|██████████| 761436/761436 [00:06<00:00, 126458.97it/s]


Check whether everything went ok

In [10]:
data[0]

{'sentence_id': 0,
 'tokens': [{'class': 'PLAIN', 'before': 'По ', 'after': 'По '},
  {'class': 'PLAIN', 'before': 'состоянию ', 'after': 'состоянию '},
  {'class': 'PLAIN', 'before': 'на ', 'after': 'на '},
  {'class': 'DATE',
   'before': '1862 год',
   'after': 'тысяча восемьсот шестьдесят второй год'},
  {'class': 'PUNCT', 'before': '.', 'after': '.'}]}

In [11]:
for i, elem in enumerate(data[:10]):
    for key in ("before", "after"):
        print(i, "".join([_[key] for _ in elem["tokens"]]))

0 По состоянию на 1862 год.
0 По состоянию на тысяча восемьсот шестьдесят второй год.
1 Оснащались латными рукавицами и сабатонами с не длинными носками.
1 Оснащались латными рукавицами и сабатонами с не длинными носками.
2 В конце 1811 года, вследствие конфликта с проезжим вельможей (графом Салтыковым) вынужден был оставить службу по личному прошению.
2 В конце тысяча восемьсот одиннадцатого года, вследствие конфликта с проезжим вельможей (графом Салтыковым) вынужден был оставить службу по личному прошению.
3 Тиберий Юлий Поллиен Ауспекс (лат. Tiberius Julius Pollienus Auspex) — римский политический деятель начала III века.
3 Тиберий Юлий Поллиен Ауспекс (лат. тибериус джулиус поллиенус оспекс) — римский политический деятель начала третьего века.
4 Севернее Дудинки и северо-восточнее Белочи, в низменной долине Неруссы — урочище Узлив.
4 Севернее Дудинки и северо-восточнее Белочи, в низменной долине Неруссы — урочище Узлив.
5 Получение информации об адресах, почтовых индексах, странах,

Looks well done. Construct Replaces now.

In [12]:
for elem in tqdm(data):
    elem["replaces"] = Replaces.from_sequences(
        [_["before"] for _ in elem["tokens"]],
        [_["after"] for _ in elem["tokens"]],
        False
    )
    for r1, r2 in zip(elem["replaces"], elem["replaces"][1:]):
        if r1.type != "E" and r1.text_from.endswith(" ") and r1.text_to.endswith(" "):
            r1.text_from = r1.text_from[:-1]
            r1.text_to = r1.text_to[:-1]
            r2.text_from = " " + r2.text_from
            r2.text_to = " " + r2.text_to

100%|██████████| 761436/761436 [00:16<00:00, 45418.81it/s]


Get rid of examples where latin or digits exist in resulting text

In [13]:
re_digits_latin = re.compile(r"[a-zA-Z\d]")
good_data = []
for elem in tqdm(data):
    if all(r.type == "E" for r in elem["replaces"]):
        continue
    is_ok = True
    for r in elem["replaces"]:
        if r.type == "E" and re.search(re_digits_latin, r.text_from):
            is_ok = False
            break
        if re.search(re_digits_latin, r.text_to):
            is_ok = False
            break
    if is_ok:
        good_data.append(elem)
len(data), len(good_data)

100%|██████████| 761436/761436 [00:02<00:00, 290752.10it/s]


(761436, 378074)

Check on how many examples we have so far and how latin and digits are distributed there.

In [14]:
stat_regs = {
    "re_digits": re.compile(r"\d"),
    "re_digits_latin": re.compile(r"[a-zA-Z\d]"),
    "re_latin": re.compile(r"[a-zA-Z]")
}
for stat_name, stat_re in stat_regs.items():
    print(
        stat_name,
        len([elem for elem in tqdm(good_data) if any(re.search(stat_re, r.text_from) for r in elem["replaces"])])
    )

100%|██████████| 378074/378074 [00:00<00:00, 597261.03it/s]


re_digits 293157


100%|██████████| 378074/378074 [00:00<00:00, 621195.20it/s]


re_digits_latin 344709


100%|██████████| 378074/378074 [00:00<00:00, 434262.38it/s]

re_latin 100163


Look of what they are.

In [15]:
for i, elem in enumerate(good_data[:10]):
    print(f'{elem["sentence_id"]}\n{elem["replaces"]}\n')

0
E|По состоянию на 
R|1862 год => тысяча восемьсот шестьдесят второй год
E|.

2
E|В конце 
R|1811 года => тысяча восемьсот одиннадцатого года
E|, вследствие конфликта с проезжим вельможей (графом Салтыковым) вынужден был оставить службу по личному прошению.

3
E|Тиберий Юлий Поллиен Ауспекс (лат. 
R|Tiberius Julius Pollienus Auspex => тибериус джулиус поллиенус оспекс
E|) — римский политический деятель начала 
R|III => третьего
E| века.

9
E|Впоследствии многие пилоты, которые были коллегами экипажа рейса 
R|254 => двести пятьдесят четыре
E|, были озадачены: как можно было допустить такую ошибку.

10
E|Полудоспех — англ. 
R|Half Armor => халф армор
E| — латная защита рук и корпуса.

11
E|в 
R|1895—1896 => тысяча восемьсот девяносто пятом тысяча восемьсот девяносто шестом
E| годах служил на Черноморском флоте на канонерской лодке «Терец».

12
E|Данная поправка была внесена на рассмотрение Съезда народных депутатов 
R|РСФСР => р с ф с р
E|.

13
E|Революция 
R|1905 года => тысяча девятьс

Save it finally

In [16]:
import json


with open("kaggle.jsonl", "w") as f:
    for elem in tqdm(good_data):
        json.dump(
            {
                "sentence_id": elem["sentence_id"],
                "replaces": elem["replaces"],
            },
            f,
            ensure_ascii=False
        )
        f.write("\n")

100%|██████████| 378074/378074 [00:04<00:00, 77999.29it/s]


In [18]:
!wc -l {PATHES["save"]}

378074 /home/jovyan/data/kaggle.jsonl


In [19]:
!ls -lh {PATHES["save"]}

-rw-r--r-- 1 jovyan users 187M Jan 17 19:59 /home/jovyan/data/kaggle.jsonl
